# Mutual Information Check With Features and Target

## 1- Initial Preprocessing

In [1]:
import numpy as np
import pandas as pd

import sys
import os
sys.path.append(os.path.abspath('../'))

from Models.LinearRegression import LinearRegression
from Utils.Preprocessor import Preprocessor
from Utils.Utils import root_mean_squared_error, train_test_split, initial_preprocessing

In [2]:
# Read the data
train = pd.read_csv('../Data/train.csv', index_col='Id')

In [3]:
# Remove unnecessary features based on exploratory data analysis part 1.
train = initial_preprocessing(train)

In [4]:
X = train.drop(columns=["num_wins_agent1", "num_draws_agent1", "num_losses_agent1", "utility_agent1"], axis=1)
y = train["utility_agent1"]
y1 = train["num_wins_agent1"]
y2 = train["num_draws_agent1"]
y3 = train["num_losses_agent1"]

In [5]:
# Preprocess the data
preprocessor = Preprocessor(normalize=True, one_hot_encode=True)

X = preprocessor.fit_transform(X)

# Convert back to pandas dataframe
X = pd.DataFrame(X, columns=preprocessor.get_column_names())

## 2- Mutual Information Check with Target

### 2.1- utility_agent1

In [6]:
import numpy as np
import pandas as pd
from collections import Counter

def calculate_mutual_information(X, y):
    def entropy(values):
        """Calculate the entropy of a dataset."""
        # For continuous variables, use binning
        if values.dtype.kind in ['f', 'i']:
            values = pd.qcut(values, q=10, labels=False, duplicates='drop')
        
        total = len(values)
        counts = Counter(values)
        probabilities = np.array([count / total for count in counts.values()])
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))
    
    def conditional_entropy(feature, target):
        """Calculate the conditional entropy of target given feature."""
        if feature.dtype.kind in ['f', 'i']:
            feature = pd.qcut(feature, q=10, labels=False, duplicates='drop')
            
        total = len(feature)
        unique_values = np.unique(feature)
        cond_entropy = 0
        
        for value in unique_values:
            indices = np.where(feature == value)[0]
            subset = target[indices]
            prob = len(indices) / total
            cond_entropy += prob * entropy(subset)
            
        return cond_entropy
    
    # Input validation
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas DataFrame")
    if not isinstance(y, (pd.Series, np.ndarray)):
        raise TypeError("y must be a pandas Series or numpy array")
    if len(X) != len(y):
        raise ValueError("X and y must have the same number of samples")
        
    # Drop categorical columns and store column names
    numeric_X = X.select_dtypes(include=['int64', 'float64'])
    colnames = numeric_X.columns
    
    # Convert to numpy arrays
    X_values = numeric_X.values
    y_values = y.values if isinstance(y, pd.Series) else y
    
    # Calculate mutual information for each feature
    mutual_info = []
    target_entropy = entropy(y_values)
    
    for i in range(X_values.shape[1]):
        feature = X_values[:, i]
        cond_entropy = conditional_entropy(feature, y_values)
        mi = target_entropy - cond_entropy
        mutual_info.append(mi)
    
    # Create a DataFrame with results
    mutual_info_df = pd.DataFrame({
        "Feature": colnames,
        "Mutual Information": mutual_info
    }).sort_values(by="Mutual Information", ascending=False)
    
    return mutual_info_df

In [7]:
mutual_info_utility_agent1 = calculate_mutual_information(X, y)

In [8]:
mutual_info_utility_agent1.head(10)

,Feature,Mutual Information
388,Balance,0.071640
391,Timeouts,0.052316
387,AdvantageP1,0.051536
73,NumContainers,0.018607
377,DurationMoves,0.016352
98,NumStartComponentsHand,0.006821
332,EliminatePiecesLoss,0.000000
283,BackwardRightDirection,0.000000
282,ForwardRightDirection,0.000000
281,LeftwardsDirection,0.000000


### 2.1- num_wins_agent1

In [9]:
mutual_info_num_wins_agent1 = calculate_mutual_information(X, y1)

In [10]:
mutual_info_num_wins_agent1.head(10)

,Feature,Mutual Information
98,NumStartComponentsHand,0.004996
282,ForwardRightDirection,0.000000
281,LeftwardsDirection,0.000000
280,LeftwardDirection,0.000000
279,BackwardsDirection,0.000000
278,ForwardsDirection,0.000000
277,BackwardDirection,0.000000
324,NoTargetPieceEnd,0.000000
339,NoOwnPiecesWin,0.000000
338,NoOwnPiecesEndFrequency,0.000000


### 2.2- num_draws_agent1

In [11]:
mutual_info_num_draws_agent1 = calculate_mutual_information(X, y2)

In [12]:
mutual_info_num_draws_agent1.head(10)

,Feature,Mutual Information
389,Completion,0.298580
390,Drawishness,0.297891
377,DurationMoves,0.258214
68,NumCentreSites,0.244361
84,NumComponentsTypePerPlayer,0.227698
392,OutcomeUniformity,0.223815
142,HopDecisionFrequency,0.216059
376,DurationActions,0.205663
391,Timeouts,0.189423
193,RemoveEffectFrequency,0.186527


### 2.3- num_losses_agent1

In [13]:
mutual_info_num_lossess_agent1 = calculate_mutual_information(X, y3)

In [14]:
mutual_info_num_lossess_agent1.head(10)

,Feature,Mutual Information
441,PieceNumberMaxIncrease,0.00267
283,BackwardRightDirection,0.00000
282,ForwardRightDirection,0.00000
281,LeftwardsDirection,0.00000
280,LeftwardDirection,0.00000
279,BackwardsDirection,0.00000
278,ForwardsDirection,0.00000
325,NoTargetPieceEndFrequency,0.00000
324,NoTargetPieceEnd,0.00000
339,NoOwnPiecesWin,0.00000
